In [1]:
from langchain_ollama import ChatOllama

# ollama pull deepseek-r1:1.5b
#llm = ChatOllama(model="deepseek-r1:1.5b")
llm = ChatOllama(model="llama3.2:1b")

/Users/hangyeong/Workspace/practice-llm/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt_template = PromptTemplate(
    template = "What is the capital of {country}? Return the name of the city only",
    input_variables = ["country"]
)

llm.invoke(prompt_template.invoke({"country" : "France"}))

output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(prompt_template.invoke({"country" : "France"})))

'Paris.'

In [3]:
from langchain_core.output_parsers import JsonOutputParser

country_detail_prompt = PromptTemplate(
    template = """Give following information about {country}:
    - Capital
    - Population
    - Language
    - Currency

    return it in JSON format. and return the JSON dictionary only.
    """,
    input_variables = ["country"]
)

country_detail_parser = JsonOutputParser()
json_response = llm.invoke(country_detail_prompt.invoke({"country" : "France"}))

print(json_response)
country_detail_parser.invoke(json_response)

content='```\n{\n  "capital": "Paris",\n  "population": "67.2 million",\n  "language": "French",\n  "currency": "Euro"\n}\n```' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-24T15:34:15.608982Z', 'done': True, 'done_reason': 'stop', 'total_duration': 932933583, 'load_duration': 109585750, 'prompt_eval_count': 62, 'prompt_eval_duration': 48702209, 'eval_count': 37, 'eval_duration': 415963585, 'logprobs': None, 'model_name': 'llama3.2:1b'} id='run--dc04dc53-38db-48a8-a7ec-37f4dbc8e588-0' usage_metadata={'input_tokens': 62, 'output_tokens': 37, 'total_tokens': 99}


{'capital': 'Paris',
 'population': '67.2 million',
 'language': 'French',
 'currency': 'Euro'}

In [4]:
# 위처럼 문자열을 JSON 형식으로 변환하는 것이 아니라, 구조화된 데이터를 반환하는 방법(안정적인 JSON 형식 반환)
from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
    capital: str = Field(description="The capital of country")
    population: int = Field(description="The population of country")
    language: str = Field(description="The language of country")
    currency: str = Field(description="The currency of country")

structured_llm = llm.with_structured_output(CountryDetail)
response = structured_llm.invoke(prompt_template.invoke({"country" : "France"}))

print(response)
print(type(response))
    

capital='Paris' population=1000000 language='French' currency='Euro'
<class '__main__.CountryDetail'>
